## Transfer learning on a pre-trained Wav2Vec2 on small datasets for Urdu-ASR:

This notebook explores the practicality of transfer learning (fine-tuning) for Automatic Speech Recognition. Our code mostly follows: [Fine-tune Wav2vec for English ASR](https://huggingface.co/blog/fine-tune-wav2vec2-english) by Patrick von Platen. Details on the Wav2Vec2 model can be found [here](https://ai.meta.com/blog/wav2vec-20-learning-the-structure-of-speech-from-raw-audio/). 

The [Wav2Vec2 model](https://github.com/facebookresearch/fairseq/tree/main/examples/wav2vec#wav2vec-20) used was pre-trained on the Librispeech corpus.

In addition, we wanted to explore the usefulness of pre-trained models on resource-starved environments. In particular we wanted to test the following:
- The feasilibity of low end PCs for this task. Our machine had the GTX 1050M as the GPU with 4GB of available VRAM. This is a low-end GPU available with many low-end and affordable laptops. However, we had to adapt our code to be able to run locally and on a low-end device.

- The practicality of fine-tuning the model on different languages. For this, we fine-tuned the model on publicly availble datasets in Urdu.

- The possibility of expansion to different languages. While English and Urdu are vastly different languages, the north-western and the northern regions of the sub-continent present dialect continuums locally and areally. The distinction between language, dialect and accent is often fuzzy. While prestige dialects and languages have enough available resources to have state-of-the-art ASR, dialects with lower prestige do not. Future work could explore the effectiveness of ASR models trained on prestige dialects in the aformentioned regions on dialects with less prestige. For this English and Urdu, while not being entirely analogous, serve as a stand-in.


### Summarizing the Methodology and Results:
- Our methodology uses gradient accumulation and a batch size of 1 to allow for the model to run on such a low-end device.

- We obtained a WER of 0.225. Given our small dataset, this is a very good result. For comparison, state-of-the-art ASR models tend to have a WER of 0.150-0.200.

### Imports:

In [1]:
import librosa
import regex as re
import pandas as pd
import random
import pyarrow as pa
from datasets import Dataset, DatasetDict, load_metric
from IPython.display import display, HTML
import json
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor, Wav2Vec2ForCTC, Trainer
import torch
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
import numpy as np
import os
import gc
import transformers
os.environ["TRANSFORMERS_OFFLINE"] = "1" # Allow transformers from huggingface to run offline.
num_speakers = 3

### Dataset Loading and Preprocessing:

#### Dataset:
Our dataset is composed of the recordings of the PRUS dataset and a compilation of news clippings. The speakers are undergraduates from LUMS.

- [The PRUS dataset](https://www.c-salt.org/downloads/prus): A compilation of 708 urdu sentences generated using a greedy approach. The goal was to obtain a dataset with all tri-phoneme combination (word-boundaries included) possible within Urdu, with each tri-phoneme equally likely. The dataset consists of naturally occuring words in Urdu, and is grammatically correct. However, the dataset is not representative of natural Urdu speech and the syntax, word-order, sentence length and the lexicon of many of the sentences is rarely or never part of natural Urdu speech. 

- Newspaper clippings: A compilation of 505 headlines and sentences from various news articles and newspapers within the last few years. Compared with the PRUS dataset, this dataset is far more representative of modern, albeit highly formal Urdu speech.

##### Transcriptions and preprocessing:

- The vocabulary consists of the Urdu alphabet plus a few other characters deemed necessary.

- The data has been pre-processed to remove any characters that do not represent a phoneme.

- Harakat (used to demarcate vowels) are not considered part of the vocabulary given that the Urdu script is a partial Abjad and the reader is expected to infer them.

- The tashdeed (used to represent gemination) is not considered part of the vocabulary given that gemination follows specific rules and is readily infered from the context.

- Digits are replaced with their word representations to facilitate the training of the model. This also means that for any new speech, any numbers spoken will be represented using words.

- In addition, a few special unicode characters are removed (\u200c and \u200e)

In [2]:
prus_transcriptions = open("./Dataset/Trasncriptions - PRUS.txt" , 'r', encoding = "UTF-8")
prus_transcriptions = prus_transcriptions.readlines()

news_transcriptions = open("./Dataset/Transcriptions - News clippings.txt", 'r', encoding = "UTF-8")
news_transcriptions = news_transcriptions.readlines()

to_ignore_regex = "[\,\?\.\!\-\;\:\"\\()'؟۔’‘،ًٌَُِّٰٔٓ]"

# PRUS Dataset
for i in range(len(prus_transcriptions)):
    prus_transcriptions[i] = prus_transcriptions[i].replace("\n","")
    prus_transcriptions[i] = re.sub(to_ignore_regex, "", prus_transcriptions[i])
    prus_transcriptions[i] = re.sub("0", "صفر", prus_transcriptions[i])
    prus_transcriptions[i] = re.sub("1", "ایک", prus_transcriptions[i])
    prus_transcriptions[i] = re.sub("2", "دو", prus_transcriptions[i])
    prus_transcriptions[i] = re.sub("3", "تین", prus_transcriptions[i])
    prus_transcriptions[i] = re.sub("4", "چار", prus_transcriptions[i])
    prus_transcriptions[i] = re.sub("5", "پانچ", prus_transcriptions[i])
    prus_transcriptions[i] = re.sub("6", "چھ", prus_transcriptions[i])
    prus_transcriptions[i] = re.sub("7", "سات", prus_transcriptions[i])
    prus_transcriptions[i] = re.sub("8", "اٹھ", prus_transcriptions[i])
    prus_transcriptions[i] = re.sub("9", "نو", prus_transcriptions[i])
    prus_transcriptions[i] = re.sub("\u200c", "", prus_transcriptions[i])
    prus_transcriptions[i] = re.sub("\u200e", "", prus_transcriptions[i])

# News Clippings Dataset
for i in range(len(news_transcriptions)):
    news_transcriptions[i] = news_transcriptions[i].replace("\n","")
    news_transcriptions[i] = re.sub(to_ignore_regex, "", news_transcriptions[i])
    news_transcriptions[i] = re.sub("0", "صفر", news_transcriptions[i])
    news_transcriptions[i] = re.sub("1", "ایک", news_transcriptions[i])
    news_transcriptions[i] = re.sub("2", "دو", news_transcriptions[i])
    news_transcriptions[i] = re.sub("3", "تین", news_transcriptions[i])
    news_transcriptions[i] = re.sub("4", "چار", news_transcriptions[i])
    news_transcriptions[i] = re.sub("5", "پانچ", news_transcriptions[i])
    news_transcriptions[i] = re.sub("6", "چھ", news_transcriptions[i])
    news_transcriptions[i] = re.sub("7", "سات", news_transcriptions[i])
    news_transcriptions[i] = re.sub("8", "اٹھ", news_transcriptions[i])
    news_transcriptions[i] = re.sub("9", "نو", news_transcriptions[i])
    news_transcriptions[i] = re.sub("\u200c", "", news_transcriptions[i])
    news_transcriptions[i] = re.sub("\u200e", "", news_transcriptions[i])

##### Audio data:

In [3]:
prus_path = "./Dataset/PRUS/"
news_path = "./Dataset/News_Clippings/"

# files = []
audios = []
transcriptions = []

# PRUS dataset
for i in range(num_speakers):
    for j in range(len(prus_transcriptions)):
        # File
        file_path = prus_path + str(j+1) + "_" + "speaker" + str(i+1) + "_prus.wav"
        # files.append(file_path)
        
        # Audio
        array, sampling_rate = librosa.load(file_path, sr=16000)
        audios.append({"array": array, "path": file_path, "sampling_rate": sampling_rate})

        # Transcriptions
        transcriptions.append(prus_transcriptions[j])

# News dataset
for i in range(num_speakers):
    for j in range(len(news_transcriptions)):
        # File
        file_path = news_path + str(j+1) + "_" + "speaker" + str(i+1) + "_news.wav"
        # files.append(file_path)
        
        # Audio
        array, sampling_rate = librosa.load(file_path, sr=16000)
        audios.append({"array": array, "path": file_path, "sampling_rate": sampling_rate})

        # Transcriptions
        transcriptions.append(news_transcriptions[j])


# Randomizing
# randomized_data = list(zip(files, audios, transcriptions))
randomized_data = list(zip(audios, transcriptions))
random.shuffle(randomized_data)

# files, audios, transcriptions = zip(*randomized_data)
audios, transcriptions = zip(*randomized_data)

randomized_data = 0

#### Train-Test Split:

The model is trained on 80% of the data and evaluated on the remaining 20%. The data is randomized before splitting.

In [7]:
dataset_size = len(audios)
train_split = 0.8
train_size = int(train_split * dataset_size)

train_files = []
train_audios = []
train_transcriptions = []

test_files = []
test_audios = []
test_transcriptions = []

# Training data
for i in range(0, train_size):
    #train_files.append(files[i])
    train_audios.append(audios[i])
    train_transcriptions.append(transcriptions[i])

# Test data
for i in range(train_size, dataset_size):
    #test_files.append(files[i])
    test_audios.append(audios[i])
    test_transcriptions.append(transcriptions[i])


# Creating the train and test datasets.
# df = pd.DataFrame({'file': train_files, 'audio': train_audios , 'text' : train_transcriptions})
df = pd.DataFrame({'audio': train_audios , 'transcription' : train_transcriptions})
train_dataset = Dataset(pa.Table.from_pandas(df))

# df = pd.DataFrame({'file': test_files, 'audio': test_audios , 'text' : test_transcriptions})
df = pd.DataFrame({'audio': test_audios , 'transcription' : test_transcriptions})
test_dataset = Dataset(pa.Table.from_pandas(df))

# Creating the overall urdu dataset used for fine-tuning the model.
ur_fine_dataset = DatasetDict({"train": train_dataset , "test": test_dataset})
ur_fine_dataset

train_dataset = 0
test_dataset = 0

#### Training dataset transcriptions after pre-processing:

In [5]:
def displayTranscriptions(dataset, samples=5):
    # Warning
    if samples >= len(dataset):
        print("Warning: can only display a maximum of", len(dataset), "samples")
        samples = len(dataset)
    # Creating indicies
    indices = []
    for _ in range(samples):
        i = random.randint(0, len(dataset)-1)
        
        while i in indices:
            i = random.randint(0, len(dataset)-1)
        
        indices.append(i)
    
    dataframe = pd.DataFrame(dataset[indices]["transcription"])
    dataframe.columns = ["Transcriptions"]
    display(HTML(dataframe.to_html())) 

In [6]:
displayTranscriptions(ur_fine_dataset["train"], 10)

,Transcriptions
0,خیال کیا جاتا ہے کہ اس طرح کی جاسوسی کا زیادہ تر نشانہ سماجی کارکن صحافی اور سیاستدان بنتے ہیں
1,بدصورت بانسری کی انوالومنٹ سے سیلرز کی ہیرو سے گراں تخریب نیچرل تھی
2,حروں نے شہابیے کو ساگر میں پھینکا اور پیداوار کے رزق کی ڈیل کرنے کے بعد گاوں سے اوجھل ہو گئے
3,سوم درجے کی جعلسازی کا بھیڑیا تھیلی اٹھایا اور اپنے پڑدادا کے جلو میں دب گیا ہے
4,غیر جانبدار افغان بھابی کے ساتھ نتھی گڑیا کا موڈ بدلیں تو خفت رفو ہو سکتی ہے
5,میلے کچیلے خام فوم کے سویٹر میں کلراٹھی چقندر باندھیں تو یہی شے خوبصورت ڈھال ہے
6,وائس آف امریکہ سے بات کرتے ہوئے ان کا کہنا تھا کہ وہ ڈرامے کے اختتام سے بالکل خوش نہیں
7,گاڑی کے نظام میں ایسی کوئی چیز نہیں ہے جو یہ تصدیق کر سکے کہ ویڈیو گیم کھیلنے والا ڈرائیور ہے یا اس کے ساتھ والی سیٹ پر بیٹھا ہوا مسافر
8,دنیا بھر میں پیٹرولیم مصنوعات کی قیمتوں میں مسلسل اضافے سے لوگ پریشان ہیں
9,گگلی کے تجربہ اور تشخیص پر ہدایت اور تعریفیں چاہتے ہیں


#### Generating the vocabulary:

The initial vocabulary consists of the Urdu alphabet plus a few other characters deemed necessary.

In [8]:
def generateVocab(dataset):
    for i in range(len(dataset)):
        text = " ".join(dataset["transcription"])
    
    return list(set(text))


vocabulary = generateVocab(ur_fine_dataset["train"])
vocabulary.append("|")
vocabulary.remove(" ")
vocabulary.append("[UNK]")
vocabulary.append("[PAD]")
print(vocabulary)

# Creating a vocabulary dictionary
vocabulary_dictionary = {}

i = 0
for char in vocabulary:
    vocabulary_dictionary[char] = i
    i += 1

vocabulary_size = len(vocabulary_dictionary)

['چ', 'ٹ', 'گ', 'پ', 'ء', 'خ', 'م', 'ت', 'ص', 'ڑ', 'ق', 'ژ', 'ں', 'ئ', 'ر', 'ع', 'ے', 'ط', 'ا', 'ڈ', 'ذ', 'ک', 'غ', 'س', 'ف', 'ج', 'ش', 'ہ', 'ن', 'ؤ', 'ب', 'ض', 'ح', 'ز', 'ظ', 'آ', 'و', 'ل', 'ی', 'ث', 'د', 'ھ', '|', '[UNK]', '[PAD]']


#### Converting to json:
The vocabulary is extracted to a json file to allow it to be used as input to the Wav2Vec2Tokenizer

In [9]:
with open("vocabulary.json", "w") as json_file:
    json.dump(vocabulary_dictionary, json_file)

### Model preparing and training:

#### Tokenizer

Initializes the tokenizer, a huggingface made class which converts the input data into tokens.

In [10]:
tokenizer = Wav2Vec2CTCTokenizer("./vocabulary.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [11]:
tokenizer.save_pretrained("./Model/final_model/")

('./Model/final_model/tokenizer_config.json',
 './Model/final_model/special_tokens_map.json',
 './Model/final_model/vocab.json',
 './Model/final_model/added_tokens.json')

#### Feature extractor

Initializes the feature extractor, a huggingface made class which prepares audio data into features to use within in a model. More information can be found [here](https://huggingface.co/docs/transformers/main_classes/feature_extractor#:~:text=A%20feature%20extractor%20is%20in,for%20audio%20or%20vision%20models.).

In [12]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [13]:
feature_extractor.save_pretrained("./Model/final_model/")

['./Model/final_model/preprocessor_config.json']

#### Processor:

Combines the tokenizer and feature extractor into one.

In [14]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

#### Pre-processing the audio data for training:

- The audio is loaded and resampled (not needed in our case)

- The input values are stored after processing. In this case, the audio data is only normalized.

- The transcriptions are encoded to label IDs.

In [15]:
def preprocessingAudio(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate = audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcription"]).input_ids
    return batch

In [16]:
ur_fine_dataset = ur_fine_dataset.map(preprocessingAudio, remove_columns = ur_fine_dataset.column_names["train"], num_proc=1)

Map:   0%|          | 0/2911 [00:00<?, ? examples/s]

d:\CS\Projects\Speech to text-Urdu\hugvenv\lib\site-packages\transformers\models\wav2vec2\processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/728 [00:00<?, ? examples/s]

#### Initializing the model:

In [36]:
@dataclass
class DataCollatorCTCWithPadding:
    
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        labels = labels.cuda()

        input_values = batch['input_values']
        input_values = input_values.cuda()

      
        batch["labels"] = labels
        batch["input_values"] = input_values

        return batch

In [37]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

# The metric used is Word Error Rate (WER) as it the most commonly used method for ASR models.
wer_metric = load_metric("wer")

#### Computing the WER on the training data:

In [38]:
def computeWER(predicted):
    predicted_logits = predicted.predictions
    predicted_ids = np.argmax(predicted_logits, axis=-1)

    predicted.label_ids[predicted.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(predicted_ids)
    
    # We do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(predicted.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

#### Loading the Wav2Vec2pretrained checkpoint:

In [39]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base", 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size = vocabulary_size
)

d:\CS\Projects\Speech to text-Urdu\hugvenv\lib\site-packages\transformers\configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['quantizer.weight_proj.weight', 'project_hid.bias', 'project_hid.weight', 'quantizer.weight_proj.bias', 'quantizer.codevectors', 'project_q.bias', 'project_q.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of

#### Fine-tuning training arguments:

In [40]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=".\Model",
  group_by_length=True,
  per_device_train_batch_size=1,
  per_device_eval_batch_size=1,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=False, # Set to True for better performance
  gradient_checkpointing=True,
  save_steps=50,
  eval_steps=50,
  logging_steps=50,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
  push_to_hub=False, # Set to false if running offline
  dataloader_pin_memory=False,
  gradient_accumulation_steps=30 # To reduce memory usage
)

#### Trainer:

In [41]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=computeWER,
    train_dataset = ur_fine_dataset["train"],
    eval_dataset = ur_fine_dataset["test"],
    tokenizer=processor.feature_extractor
)

In [42]:
torch.cuda.empty_cache()
gc.collect()
torch.cuda.memory_summary(device=None, abbreviated=False)

trainer.train()

d:\CS\Projects\Speech to text-Urdu\hugvenv\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/2910 [00:00<?, ?it/s]

d:\CS\Projects\Speech to text-Urdu\hugvenv\lib\site-packages\transformers\models\wav2vec2\processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


{'loss': 12.1758, 'learning_rate': 5e-06, 'epoch': 0.52}


  0%|          | 0/728 [00:00<?, ?it/s]

...

  0%|          | 0/728 [00:00<?, ?it/s]

{'eval_loss': 0.3078209161758423, 'eval_wer': 0.2758490280614174, 'eval_runtime': 170.7182, 'eval_samples_per_second': 4.264, 'eval_steps_per_second': 4.264, 'epoch': 29.89}


d:\CS\Projects\Speech to text-Urdu\hugvenv\lib\site-packages\transformers\models\wav2vec2\processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


{'train_runtime': 70994.548, 'train_samples_per_second': 1.23, 'train_steps_per_second': 0.041, 'train_loss': 1.0566150097502875, 'epoch': 29.99}


TrainOutput(global_step=2910, training_loss=1.0566150097502875, metrics={'train_runtime': 70994.548, 'train_samples_per_second': 1.23, 'train_steps_per_second': 0.041, 'train_loss': 1.0566150097502875, 'epoch': 29.99})

#### Saving the processor and the model to a local directory:

This saves the processor and the model for future use. The model can be further fine-tuned on additional data.

In [43]:
model.save_pretrained("./Model/final_model")

### Evaluating on the test data:

#### Reloading the model from the saved directory:

In [44]:
model = model.from_pretrained("./Model/final_model")

#### Predicting the transcriptions:

In [45]:
def testTranscriptions(test_data):
    with torch.no_grad():
        input_values = torch.tensor(test_data["input_values"]).unsqueeze(0)
        logits = model(input_values).logits

    pred_ids = torch.argmax(logits, dim=-1)
    test_data["pred"] = processor.batch_decode(pred_ids)[0]
    test_data["gold"] = processor.decode(test_data["labels"], group_tokens=False)

    return test_data

In [46]:
output = ur_fine_dataset["test"].map(testTranscriptions, remove_columns=ur_fine_dataset["test"].column_names)

Map:   0%|          | 0/728 [00:00<?, ? examples/s]

#### Computing Test WER:

In [47]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions = output["pred"], references = output["gold"])))

Test WER: 0.225


#### Saving the predicted and original transcriptions to text files:

In [48]:
def saveToText(output):
    with open("test_gold.txt", "w", encoding = "UTF-8") as file:
        for i in range(len(output["gold"])):
            file.write(output["gold"][i] + "\n")
        file.close()
    
    with open("test_pred.txt", "w", encoding = "UTF-8") as file:
        for i in range(len(output["pred"])):
            file.write(output["pred"][i] + "\n")
        file.close()


saveToText(output)